In [2]:
from yahoofinancials import YahooFinancials
import numpy as np

In [43]:
class cookFinancials(YahooFinancials):
    ticker = ''
    bshData = []
    def __init__(self, ticker):
        if isinstance(ticker, str):
            self.ticker = ticker.upper()
        else:
            self.ticker = [t.upper() for t in ticker]
        self._cache = {}
        
    def get_balanceSheetHistory(self):
        self.bshData = self.get_financial_stmts('annual', 'balance')['balanceSheetHistory']
        return self.bshData
    
    def get_BV(self):
        bv = []
        if not(self.bshData):
            self.get_balanceSheetHistory()
        for i in range(np.size(self.bshData[self.ticker])):
            date_key = list(self.bshData[self.ticker][i].keys())[0]
            bv.append(self.bshData[self.ticker][i][date_key]['totalStockholderEquity'])
        return bv   
        
    def get_BV_GR(self, bv):
        if bv[-1]>0:
            return [np.size(bv)-1, (bv[0] - bv[-1])/(bv[-1]*(np.size(bv)-1))]
        else:
            return [np.size(bv)-1, (bv[0] - bv[-1])/(bv[0]*(np.size(bv)-1))]
    
    def get_suggest_price(self, cEPS, growth, years, rRate, PE, safty):
        fEPS = cEPS*(1+growth)**years
        fPrice = fEPS*PE;
        stickerPrice = fPrice/(1+rRate)**years
        return stickerPrice, stickerPrice*safty
    
    def get_PE(self):
        return (self._stock_summary_data('trailingPE')+self._stock_summary_data('forwardPE'))/2
    
    def get_decision(self,suggestPrice, stockprice):
        if suggestPrice>stockprice:
            return 'strong buy' 
        else:
            return 'do not buy'
            
            
        

In [44]:
x = cookFinancials('ZM')

In [45]:
bv = x.get_BV()

In [46]:
bv.insert(0, x.get_book_value())
print(bv)

[1499918000, 833943000, -7439000, -26671000, -28838000]


In [47]:
bvgr = x.get_BV_GR(bv)
print(bvgr)

[4, 0.2548065960939198]


In [48]:
growth = bvgr[1]
cEPS = x.get_earnings_per_share()
years = 8;
rRate = 0.2;
safty = 0.5
PE = x.get_PE()

In [49]:
x.get_pe_ratio()
x.get_PE()

210.27823999999998

In [50]:
price=x.get_suggest_price(cEPS, growth, years, rRate, PE, safty)
print(price)

(432.5332582527885, 216.26662912639426)


In [51]:
x.get_decision(price[1],x.get_current_price())

'do not buy'